# Project: Investigate a Dataset - No-show Appointments

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description

> This dataset collects information from 100k medical appointments in Brazil focused on whether the patient showed up for their appointment or were a no-show for their appointment. This dataset also provides medical attributes for each patient.

|     Feature    |                            Description                            |
|:--------------:|:-----------------------------------------------------------------:|
| PatientID      |                    Identification of a patient                    |
| AppointmentID  |                 Identification of each appointment                |
| Gender         |                       M = Male \| F = Female                      |
| ScheduledDay   |             Date when the appointment was scheduled.              |
| AppointmentDay |                        Date of appointment                        |
| Age            |                         Age of the patient                        |
| Neigbourhood   |           Location of where the appointment takes place           |
| Scholarship    | Indicates if the patient is enrolled in Brasilian welfare program |
| Hipertension   |                    If patient has hipertension                    |
| Diabetes       |                      If patient has diabetes                      |
| Alcoholism     |                     If patient has alcoholism                     |
| Handcap        |                    If patient has special needs                   |
| SMS_received   |                If SMS reminder was sent to patient                |
| No-show        |               No: Showed up \| Yes: Did not show up hat to explore.

### Question(s) for Analysis

- Do men no-show for appointments more than women?
- Do SMS reminders decrease no-shows?
- Is there a day of the week that has more no-shows than other days?
- Which location has the least amount of no-shows?



### Import necessary libraries

In [1]:
# Remember to include a 'magic word' so that your visualizations are plotted
#   inline with the notebook. See this page for more:
#   http://ipython.readthedocs.io/en/stable/interactive/magics.html

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
# Upgrade pandas to use dataframe.explode() function. 
# !pip install --upgrade pandas==0.25.0

### Read in csv file

In [3]:
df = pd.read_csv('noshowappointments-kagglev2-may-2016.csv')

<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you **document your data cleaning steps in mark-down cells precisely and justify your cleaning decisions.**


### General Properties
> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

### Explore the dataset

In [4]:
# Find number of rows and columns as tuple (rows, columns)
df.shape

(110527, 14)

In [5]:
# Quick check on the top of the dataset
df.head(5)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [6]:
# Quick check on bottom of the dataset
df.tail(5)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
110522,2.572134e+12,5651768,F,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56,MARIA ORTIZ,0,0,0,0,0,1,No
110523,3.596266e+12,5650093,F,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51,MARIA ORTIZ,0,0,0,0,0,1,No
110524,1.557663e+13,5630692,F,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21,MARIA ORTIZ,0,0,0,0,0,1,No
110525,9.213493e+13,5630323,F,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38,MARIA ORTIZ,0,0,0,0,0,1,No
110526,3.775115e+14,5629448,F,2016-04-27T13:30:56Z,2016-06-07T00:00:00Z,54,MARIA ORTIZ,0,0,0,0,0,1,No


In [7]:
# Check info on data such as index, column names, non-null counts, and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


> No missing data to fix.
> 
> ScheduledDay and AppointmentDay columns are objects but will need to be converted to DateTime
>
> Fix display of float for readability

In [8]:
# Convert ScheduledDay and AppointmentDay to DateTime format
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])

In [9]:
# Change float values to not have any values after the decimal
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [10]:
# Get a summary statistics of the data for all the columns 
df.describe(include='all')

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
count,110527,110527,110527,110527,110527,110527,110527,110527,110527,110527,110527,110527,110527,110527
unique,NaN,NaN,2,NaN,NaN,NaN,81,NaN,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,F,NaN,NaN,NaN,JARDIM CAMBURI,NaN,NaN,NaN,NaN,NaN,NaN,No
freq,NaN,NaN,71840,NaN,NaN,NaN,7717,NaN,NaN,NaN,NaN,NaN,NaN,88208
mean,147496265710394,5675305,NaN,2016-05-09 07:49:15.846273024+00:00,2016-05-19 00:57:50.008233472+00:00,37,NaN,0,0,0,0,0,0,NaN
min,39218,5030230,NaN,2015-11-10 07:13:56+00:00,2016-04-29 00:00:00+00:00,-1,NaN,0,0,0,0,0,0,NaN
25%,4172614444192,5640286,NaN,2016-04-29 10:27:01+00:00,2016-05-09 00:00:00+00:00,18,NaN,0,0,0,0,0,0,NaN
50%,31731838713978,5680573,NaN,2016-05-10 12:13:17+00:00,2016-05-18 00:00:00+00:00,37,NaN,0,0,0,0,0,0,NaN
75%,94391720898175,5725524,NaN,2016-05-20 11:18:37+00:00,2016-05-31 00:00:00+00:00,55,NaN,0,0,0,0,0,1,NaN
max,999981631772427,5790484,NaN,2016-06-08 20:07:23+00:00,2016-06-08 00:00:00+00:00,115,NaN,1,1,1,1,4,1,NaN


> Females make up the majority of the patients with appointments
>
> Min age is -1, this will need to be cleaned up. Max age is 115 which is possible so that will be left.
>
> 2016-05-03 is the most frequent date to for an appointment to be scheduled (4,238 appointments were scheduled)
>
> 2016-06-06 is the most frequent date for an appointment to be on (4,692 appointments on this date)
>
> 81 unique Neighbourhoods and JARDIM CAMBURI has the most appointments (7,717 appointments at this location)

In [11]:
# Count duplicated rows
df.duplicated().sum()

0

In [12]:
"""
Checks for duplicate values in each column of a dataset (dupe or dupes)
Returns column name, number of duplicated values, and list of duplicated values with frequency of each value (if applicable)
"""
def check_for_dupes(data, col):
    dupe_count = data[col].duplicated().sum()
    if dupe_count == 0:
        return f'{col} has {dupe_count} dupes\n'
    elif dupe_count == 1:
        return f'{col} has {dupe_count} dupe:\n{data[col].value_counts()}\n'
    else:
        return f'{col} has {dupe_count} dupes:\n{data[col].value_counts()}\n'

In [13]:
# Print out the duplicate values for each column in the data set
for col in df.columns:
    print(check_for_dupes(df, col), '\n', '--------------------------------------------', '\n')

PatientId has 48228 dupes:
PatientId
822145925426128    88
99637671331        84
26886125921145     70
33534783483176     65
6264198675331      62
                   ..
862263621811        1
9267587227475       1
5764355598913       1
5712491287569       1
15576631729893      1
Name: count, Length: 62299, dtype: int64
 
 -------------------------------------------- 

AppointmentID has 0 dupes
 
 -------------------------------------------- 

Gender has 110525 dupes:
Gender
F    71840
M    38687
Name: count, dtype: int64
 
 -------------------------------------------- 

ScheduledDay has 6978 dupes:
ScheduledDay
2016-05-06 07:09:54+00:00    24
2016-05-06 07:09:53+00:00    23
2016-04-25 17:18:27+00:00    22
2016-04-25 17:17:46+00:00    22
2016-04-25 17:17:23+00:00    19
                             ..
2016-05-02 09:53:25+00:00     1
2016-05-30 09:12:28+00:00     1
2016-05-16 09:10:04+00:00     1
2016-05-09 10:17:48+00:00     1
2016-04-27 13:30:56+00:00     1
Name: count, Length: 103549, d

> Several patients have multiple appointments
>
> AppointmentID values are all unique

In [14]:
# Find all the unique values in each column
for col in df.columns:
    print(f'{col.upper()} has {df[col].nunique()} unique values:\n  {df[col].unique()} \n\n ---------------------------------------------------------\n')

PATIENTID has 62299 unique values:
  [2.98724998e+13 5.58997777e+14 4.26296230e+12 ... 7.26331493e+13
 9.96997666e+14 1.55766317e+13] 

 ---------------------------------------------------------

APPOINTMENTID has 110527 unique values:
  [5642903 5642503 5642549 ... 5630692 5630323 5629448] 

 ---------------------------------------------------------

GENDER has 2 unique values:
  ['F' 'M'] 

 ---------------------------------------------------------

SCHEDULEDDAY has 103549 unique values:
  <DatetimeArray>
['2016-04-29 18:38:08+00:00', '2016-04-29 16:08:27+00:00',
 '2016-04-29 16:19:04+00:00', '2016-04-29 17:29:31+00:00',
 '2016-04-29 16:07:23+00:00', '2016-04-27 08:36:51+00:00',
 '2016-04-27 15:05:12+00:00', '2016-04-27 15:39:58+00:00',
 '2016-04-29 08:02:16+00:00', '2016-04-27 12:48:25+00:00',
 ...
 '2016-06-07 07:45:16+00:00', '2016-06-07 07:38:34+00:00',
 '2016-04-27 15:15:06+00:00', '2016-05-03 07:51:47+00:00',
 '2016-05-03 08:23:40+00:00', '2016-05-03 09:15:35+00:00',
 '2016-05-

> The HANDCAP column has 5 unique values but there is no information on what those values represent. 


### Data Cleaning
- Remove columns that aren't needed to answer analysis questions
- Rename columns to improve uniformity and understanding of column data
- Add columns for additional data insights
- Remove any impossible data (i.e., age < 0)

 

In [15]:
"""
Create subset of dataset and commenting out column names not needed for this analysis
Copy() ensures Python recognizes new dataframe and not a reference to previous dataframe
"""
df = df[[# 'PatientId', 
    'AppointmentID', 'Gender', 'ScheduledDay',
    'AppointmentDay', 'Age', 'Neighbourhood',
    # 'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 
    'SMS_received', 'No-show']].copy()

# Check that columns needed for this analysis remain
df.head()


,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,SMS_received,No-show
0,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,No
1,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,No
2,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,No
3,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,No
4,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,No


As shown by the `check_for_dupes` function above there are patients with mulitple appointments. However, I decided to remove this feature from the dataset because I am concerned with whether an appointment was missed or not. Therefore, I am concerned about the attributes of an appointment, not of an individual patient.

In [16]:
# Rename columns
df = df.rename(columns={'AppointmentID':'Appointment_ID',
                        'ScheduledDay':'Scheduled_Day',
                        'AppointmentDay':'Appointment_Day',
                        'Neighbourhood':'Neighborhood',
                        'SMS_received':'SMS_Received',
                        'No-show':'Appointment_Missed'                  
                        })

# Check that columns have been renamed
df.head()

,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighborhood,SMS_Received,Appointment_Missed
0,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,No
1,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,No
2,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,No
3,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,No
4,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,No


In [17]:
# Replace values for appointment missed to represent boolean for if the appointment was missed
df['Appointment_Missed'] = df['Appointment_Missed'].replace({'No': 0, 'Yes': 1})

# Check that the values have been replaced
df.head()

,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighborhood,SMS_Received,Appointment_Missed
0,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,0
1,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0
2,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0
3,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0
4,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0


In [18]:
# Remove the time from both columns
df['Scheduled_Day'] = df['Scheduled_Day'].dt.date
df['Appointment_Day'] = df['Appointment_Day'].dt.date

# Check columns
df.head()

,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighborhood,SMS_Received,Appointment_Missed
0,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,0
1,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0
2,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0
3,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0
4,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0


In [19]:
# Create new columns to display the year for both scheduled and actual appointment day
df['Scheduled_Year'] = pd.to_datetime(df['Scheduled_Day']).dt.year
df['Appointment_Year'] = pd.to_datetime(df['Appointment_Day']).dt.year

# Check new columns
df.head()

,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighborhood,SMS_Received,Appointment_Missed,Scheduled_Year,Appointment_Year
0,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,0,2016,2016
1,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,2016,2016
2,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,2016,2016
3,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,2016,2016
4,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,2016,2016


In [20]:
# Create new columns to display the month for both scheduled and actual appointment day
df['Scheduled_Month'] = pd.to_datetime(df['Scheduled_Day']).dt.month
df['Appointment_Month'] = pd.to_datetime(df['Appointment_Day']).dt.month

# Check new columns
df.head()

,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighborhood,SMS_Received,Appointment_Missed,Scheduled_Year,Appointment_Year,Scheduled_Month,Appointment_Month
0,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,0,2016,2016,4,4
1,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,2016,2016,4,4
2,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,2016,2016,4,4
3,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,2016,2016,4,4
4,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,2016,2016,4,4


In [ ]:
# Create new columns to display dayofweek for both scheduled and actual appointment day
df['Scheduled_Day_Of_Week'] = pd.to_datetime(df['Scheduled_Day']).dt.dayofweek
df['Appointment_Day_Of_Week'] = pd.to_datetime(df['Appointment_Day']).dt.dayofweek

# Check new columns
df.head()

0 = Monday,
1 = Tuesday,
2 = Wednesday,
3 = Thursday,
4 = Friday,
5 = Saturday,
6 = Sunday

In [23]:
# Create a column to display number of days from scheduled to actual appointment
df['Appointment_Wait_In_Days'] = (pd.to_datetime(df['Appointment_Day']) - pd.to_datetime(df['Scheduled_Day'])).dt.days

# Check new column
df.head()

,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighborhood,SMS_Received,Appointment_Missed,Scheduled_Year,Appointment_Year,Scheduled_Month,Appointment_Month,Scheduled_Day_Of_Week,Appointment_Day_Of_Week,Appointment_Wait_In_Days
0,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,0,2016,2016,4,4,4,4,0
1,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,2016,2016,4,4,4,4,0
2,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,2016,2016,4,4,4,4,0
3,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,2016,2016,4,4,4,4,0
4,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,2016,2016,4,4,4,4,0


In [32]:
new_cols = ['Scheduled_Year', 'Appointment_Year', 'Scheduled_Month', 'Appointment_Month',
            'Scheduled_Day_Of_Week', 'Appointment_Day_Of_Week', 'Appointment_Wait_In_Days'
           ]

for col in new_cols:
    print(f'{col.upper()} has {df[col].nunique()} unique values:\n  {df[col].unique()} \n\n ---------------------------------------------------------\n')

SCHEDULED_YEAR has 2 unique values:
  [2016 2015] 

 ---------------------------------------------------------

APPOINTMENT_YEAR has 1 unique values:
  [2016] 

 ---------------------------------------------------------

SCHEDULED_MONTH has 8 unique values:
  [ 4  3  2  1  5 11 12  6] 

 ---------------------------------------------------------

APPOINTMENT_MONTH has 3 unique values:
  [4 5 6] 

 ---------------------------------------------------------

SCHEDULED_DAY_OF_WEEK has 6 unique values:
  [4 2 1 3 0 5] 

 ---------------------------------------------------------

APPOINTMENT_DAY_OF_WEEK has 6 unique values:
  [4 1 0 2 3 5] 

 ---------------------------------------------------------

APPOINTMENT_WAIT_IN_DAYS has 131 unique values:
  [  0   2   3   1   4   9  29  10  23  11  18  17  14  28  24  21  15  16
  22  43  30  31  42  32  56  45  46  39  37  38  44  50  60  52  53  65
  67  91  66  84  78  87 115 109  63  70  72  57  58  51  59  41  49  73
  64  20  33  34   6  35  36

> Unique values look correct for all the columns except the Appointment_Wait_In_Days as there are some negative values in that column.

#### Check where scheduled day is after appointment day

In [36]:
df.query('Scheduled_Day > Appointment_Day')

,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighborhood,SMS_Received,Appointment_Missed,Scheduled_Year,Appointment_Year,Scheduled_Month,Appointment_Month,Scheduled_Day_Of_Week,Appointment_Day_Of_Week,Appointment_Wait_In_Days
27033,5679978,M,2016-05-10,2016-05-09,38,RESISTÊNCIA,0,1,2016,2016,5,5,1,0,-1
55226,5715660,F,2016-05-18,2016-05-17,19,SANTO ANTÔNIO,0,1,2016,2016,5,5,2,1,-1
64175,5664962,F,2016-05-05,2016-05-04,22,CONSOLAÇÃO,0,1,2016,2016,5,5,3,2,-1
71533,5686628,F,2016-05-11,2016-05-05,81,SANTO ANTÔNIO,0,1,2016,2016,5,5,2,3,-6
72362,5655637,M,2016-05-04,2016-05-03,7,TABUAZEIRO,0,1,2016,2016,5,5,2,1,-1


In [45]:
# Confirm negative values match the query above
df.query('Appointment_Wait_In_Days < 0')

,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighborhood,SMS_Received,Appointment_Missed,Scheduled_Year,Appointment_Year,Scheduled_Month,Appointment_Month,Scheduled_Day_Of_Week,Appointment_Day_Of_Week,Appointment_Wait_In_Days
27033,5679978,M,2016-05-10,2016-05-09,38,RESISTÊNCIA,0,1,2016,2016,5,5,1,0,-1
55226,5715660,F,2016-05-18,2016-05-17,19,SANTO ANTÔNIO,0,1,2016,2016,5,5,2,1,-1
64175,5664962,F,2016-05-05,2016-05-04,22,CONSOLAÇÃO,0,1,2016,2016,5,5,3,2,-1
71533,5686628,F,2016-05-11,2016-05-05,81,SANTO ANTÔNIO,0,1,2016,2016,5,5,2,3,-6
72362,5655637,M,2016-05-04,2016-05-03,7,TABUAZEIRO,0,1,2016,2016,5,5,2,1,-1


While I could assume the dates for these records were inputted in reverse somehow, I do not know that for sure since there is no indication of why the Scheduled_Day would be after the Appointment_Day, therefore, I am going to remove these rows from the dataset.

In [48]:
"""
Query data for when appointment_wait_in_days is greater than or equal to 0, 
reset the index (don't keep the index column this creates), save as new dataset
"""
df = df.query('Appointment_Wait_In_Days >= 0') \
       .reset_index(drop=True) \
       .copy()

In [50]:
# Confirm appointment_wait_in_days with negative values was removed
df.query('Appointment_Wait_In_Days < 0')

,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighborhood,SMS_Received,Appointment_Missed,Scheduled_Year,Appointment_Year,Scheduled_Month,Appointment_Month,Scheduled_Day_Of_Week,Appointment_Day_Of_Week,Appointment_Wait_In_Days


In [17]:
# Query data for when age is greater than or equal to 0, reset the index (don't keep the index column this creates), save as new dataset
df = df.query('Age >= 0') \
       .reset_index(drop=True) \
       .copy()

In [18]:
# Confirm age value of -1 was removed
df.query('Age < 0').value_counts()

Series([], Name: count, dtype: int64)

In [51]:
df

,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Age,Neighborhood,SMS_Received,Appointment_Missed,Scheduled_Year,Appointment_Year,Scheduled_Month,Appointment_Month,Scheduled_Day_Of_Week,Appointment_Day_Of_Week,Appointment_Wait_In_Days
0,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,0,2016,2016,4,4,4,4,0
1,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,2016,2016,4,4,4,4,0
2,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,2016,2016,4,4,4,4,0
3,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,2016,2016,4,4,4,4,0
4,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,2016,2016,4,4,4,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110517,5651768,F,2016-05-03,2016-06-07,56,MARIA ORTIZ,1,0,2016,2016,5,6,1,1,35
110518,5650093,F,2016-05-03,2016-06-07,51,MARIA ORTIZ,1,0,2016,2016,5,6,1,1,35
110519,5630692,F,2016-04-27,2016-06-07,21,MARIA ORTIZ,1,0,2016,2016,4,6,2,1,41
110520,5630323,F,2016-04-27,2016-06-07,38,MARIA ORTIZ,1,0,2016,2016,4,6,2,1,41


<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. **Compute statistics** and **create visualizations** with the goal of addressing the research questions that you posed in the Introduction section. You should compute the relevant statistics throughout the analysis when an inference is made about the data. Note that at least two or more kinds of plots should be created as part of the exploration, and you must  compare and show trends in the varied visualizations. 



> **Tip**: - Investigate the stated question(s) from multiple angles. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables. You should explore at least three variables in relation to the primary question. This can be an exploratory relationship between three variables of interest, or looking at how two independent variables relate to a single dependent variable of interest. Lastly, you  should perform both single-variable (1d) and multiple-variable (2d) explorations.


### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.
ax = df['Neighborhood'].value_counts() \
    .head(20) \
    .plot(kind='bar', title='Top neighborhoods for appointments')
ax.set_xlabel('Neighborhood')
ax.set_ylabel('Appointment Count')

In [ ]:
ax = df['Missed_Appointment'].value_counts() \
    .plot(kind='bar', title='Missed appointment?')
ax.set_ylabel('Appointment count')

### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed in relation to the question(s) provided at the beginning of the analysis. Summarize the results accurately, and point out where additional research can be done or where additional information could be useful.

### Limitation
> **Tip**: Make sure that you are clear with regards to the limitations of your exploration. You should have at least 1 limitation explained clearly. 

> **Tip**: If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.

## Submitting your Project 

> **Tip**: Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> **Tip**: Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> **Tip**: Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

## OPTIONAL: Question for the reviewer
 
If you have any question about the starter code or your own implementation, please add it in the cell below. 

For example, if you want to know why a piece of code is written the way it is, or its function, or alternative ways of implementing the same functionality, or if you want to get feedback on a specific part of your code or get feedback on things you tried but did not work.

Please keep your questions succinct and clear to help the reviewer answer them satisfactorily. 

> **_Your question_**

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])